# Runnable Parallel Object

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [15]:
chat_template_books = ChatPromptTemplate.from_template(
'''
Suggest three of the best intermediate-level {programming language} books.
Answer only by listing the books.
''')

chat_template_projects = ChatPromptTemplate.from_template(
'''
Suggest three interesting {programming language} projects suitable for intermedidate-level programmers.
Answer only by listing the projects.
''')

chat_template_time = ChatPromptTemplate.from_template(
'''
I'am an intermediate level programmer.

Consider the following literature:
{books}

Also, consider the following projects:
{projects}

Roughly how much time would it take me to complete the literature and the projects?
'''
)

In [17]:
chat = ChatOpenAI(model_name = "gpt-4",
                  model_kwargs = {"seed": 365},
                  temperature = 0,
                  max_tokens = 500)

C:\Users\Asus\.conda\envs\Langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [5]:
string_parser = StrOutputParser()

In [6]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [7]:
chain_parallel = RunnableParallel({"books": chain_books, "projects": chain_projects})

In [8]:
chain_parallel.invoke({"programming language": "Python"})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Weather Forecasting Application using APIs\n2. Developing a Web Scraper for E-commerce Websites\n3. Creating a Personal Finance Tracker/Budgeting Tool'}

In [9]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [10]:
# batch() -> Allowed us to invoke teh same Runnable with different input values.

In [11]:
# RunnableParallel -> Allows us to execute several Runnables using identical input values.

In [12]:
%%time
chain_books.invoke({"programming language": "python"})

CPU times: total: 15.6 ms
Wall time: 2.18 s


'1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin'

In [13]:
%%time
chain_projects.invoke({"programming language": "python"})

CPU times: total: 15.6 ms
Wall time: 3.72 s


'1. Building a Weather Forecasting Application using APIs\n2. Developing a Web Scraper for E-commerce Websites\n3. Creating a Personal Finance Tracker/Budgeting Tool'

In [14]:
%%time
chain_parallel.invoke({"programming language": "python"})

CPU times: total: 62.5 ms
Wall time: 3.46 s


{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Weather Forecasting Application using APIs\n2. Developing a Personal Finance Tracker with Data Visualization\n3. Creating a Web Scraper for E-commerce Websites.'}

# Piping a RunnableParallel with Other Runnables

In [18]:
chain_time = (RunnableParallel({"books": chain_books,
                                "projects": chain_projects})
              | chat_template_time
              | chat
              | string_parser
             )

In [19]:
print(chain_time.invoke({'programming language': "Python"}))

The time it would take to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate each day. 

However, as a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, if you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 weeks to complete.

For the projects:

1. Building a Weather Forecasting Application using APIs - Depending on the complexity of the application and your famili

In [20]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            